In [ ]:
%load_ext autoreload
%autoreload 2

# Data formatting

In [117]:
import matplotlib.pyplot as plt
from bson import ObjectId
import pandas as pd
import numpy as np
import requests
import json
import os

In [118]:
from modules.database_tables.Orders import Orders
from modules.database_tables.Products import Products
from modules.database_tables.TaskCards import TaskCards

In [ ]:
history_days = 1000
data_url = f"https://premo.gshub.nl/api/dinand/orders/{history_days}"
data_url

In [120]:
# with open("./data/dataset.json", "r") as f:
#     data = json.load(f)
req = requests.get(data_url)
data = json.loads(req.text)

In [121]:
username = os.getenv('mongodb_user')
password = os.getenv('mongodb_password')

In [122]:
orders = Orders(username, password)
products = Products(username, password)
taskcards = TaskCards(username, password)

In [147]:
# Assuming MongoDB collections 'orders', 'products', and 'taskcards' are available

# Pre-fetch all existing orders and products IDs once
existing_orders = {doc['order_id']: doc['_id'] for doc in orders.get_orders()}
existing_products = {doc['order_id']: doc['_id'] for doc in products.get_products()}

In [ ]:
for order in data:
    if len(order['products']['task_cards']) == 0:
        continue

    # Normalize order and drop unnecessary product fields
    orderDF = pd.json_normalize(order)
    orderDF.drop(columns=[col for col in orderDF.columns if col.startswith("products.")], inplace=True)
    orderDF.rename(columns={'id': 'order_id'}, inplace=True)
    orderDF['delivery_date'] = pd.to_datetime(orderDF['delivery_date'], errors='coerce')
    orderDF['order_created_at'] = pd.to_datetime(orderDF['order_created_at'], errors='coerce')
    
    # Convert DataFrame to dictionary format
    order_dict = orderDF.to_dict(orient='records')[0]
    order_id = existing_orders.get(order_dict['order_id'])

    if order_id is None:
        # If order doesn't exist, insert it and store the inserted ID
        order_id = orders.insert_order(order_dict)
        existing_orders[order_dict['order_id']] = order_id
    
    # Prepare product details (excluding task_cards) and check if it already exists
    products_dict = {'order_id': order_id, **{k: v for k, v in order['products'].items() if k != 'task_cards'}}
    product_id = existing_products.get(order_id)

    if product_id is None:
        # If product doesn't exist, insert it and store the inserted ID
        product_id = products.insert_product(products_dict)
        existing_products[order_id] = product_id

    # Normalize task cards, insert product_id and convert timestamps
    tasksDF = pd.json_normalize(order['products'], 'task_cards')
    tasksDF.insert(0, 'product_id', product_id)
    tasksDF['task_time_start'] = pd.to_datetime(tasksDF['task_time_start'], errors='coerce')
    tasksDF['task_time_end'] = pd.to_datetime(tasksDF['task_time_end'], errors='coerce')
    
    tasks_dict = tasksDF.to_dict(orient='records')
    
    # Insert task cards if not already present
    if not taskcards.find({'product_id': product_id}):
        taskcards.insert_taskcards(tasks_dict)

In [ ]:
order_id = ObjectId('66f176004deaf93907993c62')
order = orders.find({'_id': order_id})
order

In [ ]:
product = products.find({'order_id': order_id})
product_id = ObjectId(product['_id'])
product

In [ ]:
tasks = taskcards.find_many({'product_id': product_id})
tasks